# FlowGraph with STARFit Reservoir: Big Sandy Reservoir
1. Plot the plot, show where the reservoir should be
2. Run the FGR simulation, look at flows out of where reservoir should be
3. Insert the STARFit reservoir, compare flows.

? Where will the FGR data reside? as release assets?

In [ ]:
from copy import deepcopy
import pathlib as pl
import pickle
from pprint import pprint
from shutil import rmtree

import jupyter_black
import numpy as np
from tqdm.auto import tqdm
import xarray as xr

import hvplot.xarray  # noqa, after xr

import pywatershed as pws
from pywatershed.plot import DomainPlot
from pywatershed.constants import __pywatershed_root__ as repo_root
from pywatershed.constants import zero

plot_height = 600
plot_width = 1000

jupyter_black.load()

In [ ]:
nb_output_dir = pl.Path("./06_flow_graph_starfit")
if not nb_output_dir.exists():
    nb_output_dir.mkdir()

## Big Sandy Reservoir

In [ ]:
sf_data_dir = pl.Path("/Users/jmccreight/usgs/data/starfit_datasets/")
grand_file = sf_data_dir / "GRanD_Version_1_3/GRanD_reservoirs_v1_3.shp"
istarf_file = sf_data_dir / "ISTARF-CONUS.csv"
sf_params = pws.parameters.StarfitParameters.from_istarf_conus_grand(
    grand_file=grand_file, istarf_file=istarf_file
)

In [ ]:
grand_names = sf_params.parameters["GRanD_NAME"].tolist()
big_sandy_index = [
    ii for ii, nn in enumerate(grand_names) if "big sandy" in str(nn).lower()
][0]
big_sandy_grand_id = sf_params.parameters["grand_id"][big_sandy_index]

In [ ]:
# get a parameter set with just the big sandy dike
sf_params = pws.parameters.StarfitParameters.from_istarf_conus_grand(
    grand_file=grand_file, istarf_file=istarf_file, grand_ids=[big_sandy_index]
)

In [ ]:
start_lat = sf_params.parameters["LAT_DD"]
start_lon = sf_params.parameters["LONG_DD"]
# unfortunately the above are for a different reservoir,
# TODO: is the polygon correct?
# the coords are easy to get on google maps
start_lat = 42.25547378652696
start_lon = -109.43063080023737

In [ ]:
domain_dir = pl.Path("/Users/jmccreight/usgs/data/pynhm/fgr")
domain_gis_dir = domain_dir / "GIS"

control_file = domain_dir / "nhm.control"

shp_file_hru = domain_gis_dir / "model_nhru.shp"
shp_file_seg = domain_gis_dir / "model_nsegment.shp"

In [ ]:
# add GRanD shp file? or add to the object afterwards? option to get polygons
# for sf_params above? but how to show connectivity?
_ = DomainPlot(
    hru_shp_file=shp_file_hru,
    segment_shp_file=shp_file_seg,
    start_lat=start_lat,
    start_lon=start_lon,
    start_zoom=13,
    width=1300,
    height=800,
)

From the above, by mousing over the segments we can see the reservoir should be inserted above nhm_seg 44426 and below nhm_segs 44434 and 44435. 

In [ ]:
from IPython.display import Image

display(Image("big_sandy_satellite.png", width=400))
display(Image(url="IMG_3249.jpg"))

## Flaming Gorge Domain run with NHM and NO RESERVOIR
Big Sandy Dike is upstream of the Flaming Gorge and we run it as part of that domain. Let's first take a look at the flows on segment 44426 with no reservoir present.

In [ ]:
control = pws.Control.load_prms(control_file, warn_unused_options=False)
control.edit_n_time_steps(365 * 2)
parameter_file = domain_dir / control.options["parameter_file"]
params = pws.parameters.PrmsParameters.load(parameter_file)

In [ ]:
# # run just once
# cbh_nc_dir = domain_dir
# cbh_files = [
#     domain_dir / "prcp.cbh",
#     domain_dir / "tmax.cbh",
#     domain_dir / "tmin.cbh",
# ]

# params = pws.parameters.PrmsParameters.load(domain_dir / "myparam.param")

# for cbh_file in cbh_files:
#     out_file = cbh_nc_dir / cbh_file.with_suffix(".nc").name
#     pws.utils.cbh_file_to_netcdf(cbh_file, params, out_file)

In [ ]:
nhm_processes = [
    pws.PRMSSolarGeometry,
    pws.PRMSAtmosphere,
    pws.PRMSCanopy,
    pws.PRMSSnow,
    pws.PRMSRunoff,
    pws.PRMSSoilzone,
    pws.PRMSGroundwater,
    pws.PRMSChannel,
]

# we'll use the to-channel fluxes later when running FlowGraph as a post-process
control.options["netcdf_output_var_names"] = [
    "seg_outflow",
    "sroff_vol",
    "ssres_flow_vol",
    "gwres_flow_vol",
]
run_dir = nb_output_dir / "fgr_nhm"

control.options = control.options | {
    "input_dir": domain_dir,
    "budget_type": "error",
    "calc_method": "numba",
    "netcdf_output_dir": run_dir,
}

In [ ]:
%%time
if not run_dir.exists():
    # must delete the run dir to re-run
    run_dir.mkdir()
    nhm = pws.Model(
        nhm_processes,
        control=control,
        parameters=params,
    )
    nhm.run(finalize=True)
    nhm.finalize()

In [ ]:
outflow = xr.open_dataarray(run_dir / "seg_outflow.nc").sel(nhm_seg=44426)

In [ ]:
outflow.hvplot(
    width=plot_width,
    height=plot_height,
)

## FlowGraph in Model

In [ ]:
params_file_channel = domain_dir / "parameters_PRMSChannel.nc"
params_channel = pws.parameters.PrmsParameters.from_netcdf(params_file_channel)

dis_file = domain_dir / "parameters_dis_hru.nc"
dis_hru = pws.Parameters.from_netcdf(dis_file, encoding=False)

dis_both_file = domain_dir / "parameters_dis_both.nc"
dis_both = pws.Parameters.from_netcdf(dis_both_file, encoding=False)

In [ ]:
control = pws.Control.load_prms(control_file, warn_unused_options=False)
control.edit_n_time_steps(365 * 2)
run_dir = nb_output_dir / "fgr_starfit"
control.options = control.options | {
    "input_dir": domain_dir,
    "budget_type": "error",
    "calc_method": "numba",
    "netcdf_output_dir": run_dir,
    "netcdf_output_var_names": ["node_outflows", "node_upstream_inflows"],
}

In [ ]:
nhm_processes = [
    pws.PRMSSolarGeometry,
    pws.PRMSAtmosphere,
    pws.PRMSCanopy,
    pws.PRMSSnow,
    pws.PRMSRunoff,
    pws.PRMSSoilzone,
    pws.PRMSGroundwater,
]

model_dict = {
    "control": control,
    "dis_both": dis_hru,
    "dis_hru": dis_both,
    "model_order": [],
}

for proc in nhm_processes:
    # this is the class name
    proc_name = proc.__name__
    # the processes can have arbitrary names in the model_dict and
    # an instance should not have capitalized name anyway (according to
    # python convention), so rename from the class name
    proc_rename = "prms_" + proc_name[4:].lower()
    # each process has a dictionary of information
    model_dict["model_order"] += [proc_rename]
    model_dict[proc_rename] = {}
    # alias to shorten lines below
    proc_dict = model_dict[proc_rename]
    # required key "class" specifys the class
    proc_dict["class"] = proc
    # the "parameters" key provides an instance of Parameters
    proc_param_file = domain_dir / f"parameters_{proc_name}.nc"
    proc_dict["parameters"] = pws.Parameters.from_netcdf(proc_param_file)
    # the "dis" key provides the name of the discretizations
    # which we'll supply shortly to the model dictionary
    if proc_rename == "prms_channel":
        proc_dict["dis"] = "dis_both"
    else:
        proc_dict["dis"] = "dis_hru"

In [ ]:
pprint(model_dict, sort_dicts=False)

In [ ]:
model_dict = pws.prms_channel_flow_graph_to_model_dict(
    model_dict=model_dict,
    prms_channel_dis=dis_both,
    prms_channel_dis_name="dis_both",
    prms_channel_params=params_channel,
    new_nodes_maker_dict={
        "starfit": pws.hydrology.starfit.StarfitFlowNodeMaker(None, sf_params)
    },
    new_nodes_maker_names=["starfit"],
    new_nodes_maker_indices=[0],
    new_nodes_flow_to_nhm_seg=[44426],
    graph_budget_type="warn",  # move to error
)

In [ ]:
%%time
if not run_dir.exists():
    run_dir.mkdir()
    model = pws.Model(model_dict)
    model.run()
    model.finalize()

In [ ]:
wh_44426 = np.where(params.parameters["nhm_seg"] == 44426)[0]
outflow_nodes = xr.open_dataarray(run_dir / "node_outflows.nc")[
    :, wh_44426
].drop_vars("node_coord")

In [ ]:
xr.merge([outflow, outflow_nodes]).rename(
    {"seg_outflow": "NHM", "node_outflows": "STARFIT"}
).hvplot(
    width=plot_width,
    height=plot_height,
    ylabel="streamflow (cfs)",
)

## FlowGraph as a post-process

In [ ]:
control = pws.Control.load_prms(control_file, warn_unused_options=False)
control.edit_n_time_steps(365 * 2)
run_dir = nb_output_dir / "fgr_starfit_post"
control.options = control.options | {
    "input_dir": domain_dir,
    "budget_type": "error",
    "calc_method": "numba",
    "netcdf_output_dir": run_dir,
    "netcdf_output_var_names": ["node_outflows", "node_upstream_inflows"],
}

params_file_channel = domain_dir / "parameters_PRMSChannel.nc"
params_channel = pws.parameters.PrmsParameters.from_netcdf(params_file_channel)

# dis_file = domain_dir / "parameters_dis_hru.nc"
# dis_hru = pws.Parameters.from_netcdf(dis_file, encoding=False)
if "dis_hru" in locals().keys():
    del dis_hru

dis_both_file = domain_dir / "parameters_dis_both.nc"
dis_both = pws.Parameters.from_netcdf(dis_both_file, encoding=False)

In [ ]:
sfp_ds = sf_params.to_xr_ds().copy()
sfp_ds["GRanD_CAP_MCM"] *= 6
sf_params_new = pws.Parameters.from_ds(sfp_ds)

In [ ]:
input_dir = nb_output_dir / "fgr_nhm"  # use the output of the NHM run

flow_graph = pws.prms_channel_flow_graph_postprocess(
    control=control,
    prms_channel_params=params_channel,
    prms_channel_dis=dis_both,
    input_dir=input_dir,
    new_nodes_maker_dict={
        "starfit": pws.hydrology.starfit.StarfitFlowNodeMaker(
            None, sf_params_new
        ),
        "pass_through": pws.hydrology.pass_through_node.PassThroughNodeMaker(),
    },
    new_nodes_maker_names=["starfit", "pass_through"],
    new_nodes_maker_indices=[0, 0],
    new_nodes_flow_to_nhm_seg=[44426, 44418],
)

In [ ]:
%%time
if not run_dir.exists():
    run_dir.mkdir()
    flow_graph.initialize_netcdf()
    for istep in tqdm(range(control.n_times)):
        control.advance()
        flow_graph.advance()
        flow_graph.calculate(1.0)
        flow_graph.output()

    flow_graph.finalize()

In [ ]:
wh_44426 = np.where(params.parameters["nhm_seg"] == 44426)[0]
outflow_nodes_post = (
    xr.open_dataarray(run_dir / "node_outflows.nc")[:, wh_44426]
    .drop_vars("node_coord")
    .rename("node_outflows_post")
)

In [ ]:
xr.merge(
    [
        outflow,
        outflow_nodes,
        outflow_nodes_post,
    ]
).rename(
    {
        "seg_outflow": "NHM",
        "node_outflows": "STARFIT",
        "node_outflows_post": "STARFIT CAP*6",
    }
).hvplot(
    width=950,
    height=400,
    ylabel="streamflow (cfs)",
)